# Where the data science jobs are? (Part 1) 

In this post, we're going to retrieve and clean salary data for data science related jobs. In the next installment, [Where the data science jobs are (part 2)](https://github.com/sedeh/github.io/blob/master/projects/where_the_data_science_related_jobs_are_part2.ipynb), we'll analyze the data and gain insights into data science related jobs. 

We're going to source the data from the U.S. Department of Labor (DOL). When a U.S. company wants to hire a non-U.S. worker, the company is required to file a work visa (H1B) or permanent residency (greencard) application with DOL. As part of the application, the company must disclose how much it will pay the non-U.S. worker. The company is also required to disclose the `Prevailing Wage` for the job, i.e. how much U.S. workers in similar positions are being paid. The idea is to ensure that hiring non-U.S. workers does not negatively affect the job market for U.S. workers. 

We'll take advantage of this publicly available data to explore information about data science related jobs. For this analysis, we're going to consider H1B work visa applications from 2002 to 2015.

## Set up

First, let's import the packages we'll need and set things up.

In [1]:
import re
import time
import platform
import numpy as np
import pandas as pd
import multiprocessing as mp
from urllib.request import urlopen

# For now, let's turn off panda's warning
pd.options.mode.chained_assignment = None 

# Snippets like this will be used thruout the code to get idea of program runtime
total_time = time.time() 

Let's also get some system info to help with your troubleshooting in the likely event you're using a different environment from the one I used in developing and testing this code.

In [2]:
!python --version
print(platform.platform())
print("cpu cores: {0}".format(mp.cpu_count()))

Python 3.5.2 :: Anaconda 4.1.1 (64-bit)
Linux-3.13.0-74-generic-x86_64-with-debian-jessie-sid
cpu cores: 8


## Load data

The Python script to download the data is [here](https://github.com/sedeh/github.io/blob/master/resources/h1bdata_download.py). The data is **561MB**. Let's read it in and see what it looks like.

In [3]:
h1bdataDF = pickle.load(open('h1bdataDF.pkl', 'rb'))
print("\n")
print("h1bdataDF has {0} rows and {1} columns".format(h1bdataDF.shape[0], h1bdataDF.shape[1]))
print("\n")
print(h1bdataDF.head(5))



h1bdataDF has 3886329 rows and 14 columns


        Submitted_Date      Case_Number                      Employer_Name  \
0    10/1/2006 0:03:29  I-06274-2831343     Solar Link International, Inc.   
1    10/1/2006 1:47:12  I-06274-2831345           Friendship Corner School   
2   10/3/2006 13:18:55  I-06274-2831347  The Underground World Corporation   
3  10/23/2006 20:11:19  I-06274-2831349                       Solutia Inc.   
4    10/1/2006 8:07:17  I-06274-2831351        Trustmark Insurance Company   

  Employer_City Employer_State Employer_Postal_Code  \
0       Ontario             CA                91761   
1      Bartlett             IL                60103   
2     Royal Oak             MI                48067   
3     St. Louis             MO                63141   
4   Lake Forest             IL                60045   

                                   Job_Title Approval_Status  Wage_Rate  \
0                                 Accountant       Certified  $36000.00   
1   

## Filter data

So we've about 4 million records. Let's remove any duplicate application that may be in the record.

In [4]:
h1bdataDF2 = h1bdataDF.drop_duplicates(['Case_Number'], keep='last')
print("\n")
print("h1bdataDF2 has {0} rows and {1} columns".format(h1bdataDF2.shape[0], h1bdataDF2.shape[1]))



h1bdataDF2 has 3788868 rows and 14 columns


Let's see the available options for `Approval_Status` field. 

In [5]:
h1bdataDF.Approval_Status.unique()

array(['Certified', 'Denied', 'Hold', 'Pending', None, 'DENIED',
       'CERTIFIED', 'WITHDRAWN', 'CERTIFIED-WITHDRAWN',
       'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED', 'REJECTED',
       'INVALIDATED'], dtype=object)

For our purpose, it makes sense to keep only applications that were `certified`. A `denied` application for example does not give you a lot of confidence about the quality of the application. Before we do that, we need to first remove rows with `NA`s in the `Approval_Status` column.

In [6]:
filteredDF = h1bdataDF2.dropna(subset=['Approval_Status'])
print("filteredDF: {0}".format(filteredDF.shape))
certifiedDF = filteredDF[filteredDF.Approval_Status.str.lower().str.contains('certified')]
print("certifiedDF: {0}".format(certifiedDF.shape))

filteredDF: (3788867, 14)
certifiedDF: (3550156, 14)


## Categorize data science related jobs

Now, we're ready to select applications filed for data science related jobs. I chose the selection criteria based on my view and research of the data science field. But this isn't perfect and feel free to swap in your own criteria :).

First, let's make sure `Job_Title` column contains only `string`s.

In [7]:
rows_with_strings  = certifiedDF[["Job_Title"]].apply(
       lambda row : 
          any([ isinstance(e, str) for e in row ])
       , axis=1) 
    
df_with_no_strings = certifiedDF[["Job_Title"]][~rows_with_strings]
print(df_with_no_strings.head())

       Job_Title
435289   66007.9
457835     75483
473113     27.89
494274     71400
546898    541613


It seems some job titles were entered as `float`s. Let's clean out all `float` job titles. 

In [8]:
certifiedDF2 = certifiedDF[rows_with_strings]
certifiedDF2.shape

(3550104, 14)

Now we can standardize the job titles.

In [9]:
start_time = time.time()

titles_dict = {'data analyst': 'data analyst', 'business analyst':'business analyst',
               'informatic': 'informatics', 'operations research':'operations research',
               'data scien': 'data scientist', 'intelligence': 'business intelligence', 
               'analytics': 'business intelligence', 'data consultan': 'business intelligence', 
               'data manage': 'business intelligence','reporting & analysis': 'business intelligence',
               'data min': 'business intelligence','reporting and analysis': 'business intelligence',
               'database analyst':'data architect/engineer', 'data architect': 'data architect/engineer',
               'data engineer': 'data architect/engineer','data warehous': 'data architect/engineer',
               'database admin': 'data architect/engineer', 'datawarehous': 'data architect/engineer', 
               'data model': 'data architect/engineer','etl developer':'data architect/engineer', 
               'market research analyst': 'market analyst', 'market analyst': 'market analyst', 
               'customer insight': 'market analyst', 'market insight': 'market analyst',
               'consumer insight': 'market analyst', 'marketing insight': 'market analyst',
               'insights analyst': 'market analyst', 'analytic insight': 'market analyst',
               'strategy and insight': 'market analyst', 'mkt. res. analyst': 'market analyst',
               'marketing & insights': 'market analyst', 'global insights': 'market analyst',
               'insights & analytic':'market analyst', 'strategy and insight': 'market analyst',
               'marketing & insight': 'market analyst','insights & analytic':'market analyst',
               'health care analyst':'market analyst', 'healthcare analyst': 'market analyst',
               'quantitative analyst':'market analyst', 'financial analyst':'market analyst',
               'marketing analyst':'market analyst', 'management analyst':'market analyst', 
               'business development': 'market analyst', 'consumer insight':'market analyst',
               'analytic insight':'market analyst', 'data strateg':'market analyst',
               'decision scien':'data scientist'}

titles_list = []

for index, row in certifiedDF2.iterrows():
    title = row.Job_Title.lower()
    currentTitle = [val for key,val in titles_dict.items() if key in title]
    if currentTitle:
        titles_list.append(currentTitle[0])
    else:
        titles_list.append("non data science related")
        
certifiedDF2["Job_Title_revised"] = titles_list

# Filter out non data science jobs
dsJobsDF = certifiedDF2[certifiedDF2.Job_Title_revised != "non data science related"]
dsJobsDF.reset_index(drop=True, inplace=True)

print("dsJobsDF shape: {0}".format(dsJobsDF.shape))
print("Time to select data science related jobs and standardize job titles --- %s seconds ---" % (time.time() - start_time))

dsJobsDF shape: (209240, 15)
Time to select data science related jobs and standardize job titles --- 354.9449350833893 seconds ---


## Standardize salary

Let's look at `Wage_Rate` column and see if any standardization is necessary. 

In [10]:
print(dsJobsDF.Wage_Rate_Unit.str.lower().unique())

['year' 'hour' 'month' 'week' '2 weeks' 'bi-weekly' 'yr' 'hr' 'wk' 'mth'
 'bi']


Wage rates range from hourly, weekly, bi-weekly, monthly to yearly. Let's standardize to yearly. But we need to make sure that the wage columns are clean and do not contain special characters.

In [11]:
print(dsJobsDF.Wage_Rate.unique())
print(dsJobsDF.Prevailing_Wage.unique())

['$48000.00' '$47359.00' '$48600.00' ..., '147014 -' '47819.00 -' '64501 -']
['$40706.00' '$37149.00' '$37669.00' ..., 117915.0 43.54 147014.0]


As you can see, the wage columns contains some special characters such as dollar sign and hyphen. Let's clean those out.

In [12]:
start_time = time.time()

dsJobsDF2 = dsJobsDF

# Ensure the columns are of string before applying regex
dsJobsDF2['Wage_Rate'] = dsJobsDF2['Wage_Rate'].astype(str)
dsJobsDF2['Prevailing_Wage'] = dsJobsDF2['Prevailing_Wage'].astype(str)

# Remove '$' and '-' signs from Wage_Rate column
dsJobsDF2['Wage_Rate'] = dsJobsDF2['Wage_Rate'].map(lambda x: re.sub('\$|-.*', '', x))
# Remove '$' and '-' signs from Prevailing_Wage column
dsJobsDF2['Prevailing_Wage'] = dsJobsDF2['Prevailing_Wage'].map(lambda x: re.sub('\$|-.*', '', x))
        
# Convert the cleaned wage columns to float
dsJobsDF2[['Wage_Rate','Prevailing_Wage']] = dsJobsDF2[['Wage_Rate','Prevailing_Wage']].apply(pd.to_numeric)
print(dsJobsDF2.shape)
print("Time to clean wage columns --- %s seconds ---" % (time.time() - start_time))

(209240, 15)
Time to clean wage columns --- 1.0070388317108154 seconds ---


Let's see what the wage columns look like now.

In [13]:
print(dsJobsDF2.Prevailing_Wage.unique())

[  4.07060000e+04   3.71490000e+04   3.76690000e+04 ...,   1.17915000e+05
   4.35400000e+01   1.47014000e+05]


In [14]:
print(dsJobsDF2.Wage_Rate.unique())

[  4.80000000e+04   4.73590000e+04   4.86000000e+04 ...,   4.35400000e+01
   5.05000000e+03   1.47014000e+05]


It appears we successfully removed all special characters in the wage columns. Now, finally, we can standardize wage to yearly.

In [15]:
start_time = time.time()

salary_offered = []
salary_prevailing = []
for index, row in dsJobsDF2.iterrows():
    if row.Wage_Rate_Unit.lower()[0] == 'y':
        salary_offered.append(row.Wage_Rate)
        salary_prevailing.append(row.Prevailing_Wage)
    elif row.Wage_Rate_Unit.lower()[0] == 'm':
        salary_offered.append(row.Wage_Rate * 12)
        salary_prevailing.append(row.Prevailing_Wage * 12)
    elif row.Wage_Rate_Unit.lower() in ['bi', '2 weeks', 'bi-weekly']:
        salary_offered.append(row.Wage_Rate * 26) # Assumes 52 work wks/yr
        salary_prevailing.append(row.Prevailing_Wage * 26)
    elif row.Wage_Rate_Unit.lower()[0] == 'w':
        salary_offered.append(row.Wage_Rate * 52) # Assumes 52 work wks/yr
        salary_prevailing.append(row.Prevailing_Wage * 52)
    else:
        salary_offered.append(row.Wage_Rate * 2080) # This handles the hourly rates. Assumes 52 work wks of 40 hrs/wk
        salary_prevailing.append(row.Prevailing_Wage * 2080)

dsJobsDF3 = dsJobsDF2
dsJobsDF3["Salary_Offered"] = salary_offered
dsJobsDF3["Salary_Prevailing"] = salary_prevailing

# Let's remove any rows without salary Wage information
dsJobsDF3 = dsJobsDF3.dropna(subset=['Salary_Offered'])
dsJobsDF3 = dsJobsDF3.dropna(subset=['Salary_Prevailing'])

print("Time to standardize salary --- %s seconds ---" % (time.time() - start_time))
print('dsJobsDF3: {0}'.format(dsJobsDF3.shape))
dsJobsDF3.reset_index(drop=True, inplace=True) # Resets row index so they start from 0

Time to standardize salary --- 27.00463628768921 seconds ---
dsJobsDF3: (209232, 17)


Some salary information seems questionable. We've many wages that seem unbelievably high. In the data frame below, row **2801** has hourly rate of **4330560.0**. At the other extreme, we've annual salary of **23.37** for row **24343**. These values are likely entry errors and we should remove them. 

In [16]:
print(dsJobsDF3[dsJobsDF3.Salary_Offered > 1000000].head())
print("\n\n")
print(dsJobsDF[dsJobsDF.Salary_Offered < 100].head())

           Submitted_Date      Case_Number  \
368    10/9/2006 17:12:11  I-06282-2844381   
408   10/10/2006 14:16:26  I-06283-2845897   
1865  11/15/2006 19:05:54  I-06319-2902685   
2636   12/7/2006 18:24:27  I-06341-2931261   
2801  12/13/2006 14:54:25  I-06347-2938439   

                             Employer_Name Employer_City Employer_State  \
368        JLT AEROSPACE(NORTH AMERICA)INC        LATHAM             NY   
408               Anant B. Soni M.D., P.A.       Raleigh             NC   
1865  Kismet Real Estate Investments, Inc.      Moorpark             CA   
2636          LG Electronics Alabama, Inc.    Huntsville             AL   
2801                       baik kwang corp     carlstadt             NJ   

     Employer_Postal_Code                                       Job_Title  \
368                 12110  Director of International Business Development   
408                 27612                          Database Administrator   
1865                93021                

In [17]:
# Let's keep salaries that are greater than 20,000 but less than 1.5 million
dsJobsDF3 = dsJobsDF3[(dsJobsDF3.Salary_Offered > 20000) & (dsJobsDF3.Salary_Offered < 1500000)]
dsJobsDF3 = dsJobsDF3[(dsJobsDF3.Salary_Prevailing > 20000) & (dsJobsDF3.Salary_Prevailing < 1500000)]
dsJobsDF3.shape

(208610, 17)

## Add price parity information

It's a good idea to adjust salary to reflect regional price parity (inflation). Let's get regional inflation data from the Bureau of Economic Analysis, www.bea.gov.

In [18]:
start_time = time.time()

url = 'https://www.bea.gov/newsreleases/regional/rpp/2015/xls/rpp0615.xlsx'
response = urlopen(url)
parityDF = pd.read_excel(response, skiprows=3)
parityDF.drop(parityDF.columns[[1,2,3,4,5,6,9]], axis=1, inplace=True)
parityDF.columns = ["State", "2012", "2013"]
parityDF['Price_Deflator'] = (parityDF['2012'] + parityDF['2013']) / 2 
parityDF.drop(parityDF.columns[[1,2]], axis=1, inplace=True)

print("parityDF shape: {0}".format(dsJobsDF.shape))
print("Time to read and clean BEA data --- %s seconds ---" % (time.time() - start_time))

parityDF shape: (209240, 17)
Time to read and clean BEA data --- 0.8698716163635254 seconds ---


Here's what the price parity data looks like.

In [19]:
print(parityDF.head())

        State  Price_Deflator
0     Alabama           93.65
1      Alaska          113.15
2     Arizona          103.80
3    Arkansas           93.30
4  California          120.00


We're almost ready to merge the price parity information with the salary data. The common column between both data frames is `State` and we should take a look at the `State` columns in both data frames to make sure they match. 

In [20]:
print(dsJobsDF3.Work_State.unique())

['TX' 'AL' 'FL' 'IL' 'PA' 'NY' 'MO' 'NJ' 'LA' 'CA' 'OK' 'WA' 'MA' 'WI' 'CO'
 'MN' 'NC' 'VA' 'GA' 'MD' 'OH' 'VI' 'TN' 'CT' 'MI' 'AZ' 'RI' 'NH' 'DC' 'NM'
 'NV' 'SC' 'HI' 'KY' 'NE' 'DE' 'ID' 'AR' 'MT' 'OR' 'IN' 'IA' 'ME' 'KS' 'AK'
 'SD' 'MS' 'PR' 'ND' 'UT' 'GU' 'WY' 'VT' 'WV' 'FM' 'PW' 'AS' 'MH' 'MP']


In [21]:
print(parityDF.State.unique())

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' nan 'Maximum' 'Minimum'
 'Range' 'Source: U.S. Bureau of Economic Analysis']


Notice two things. First, the price parity dataset contains extraneous rows named `maximum`, `minimum`, `range` and `nan`. Second, the two datasets use different formats for state names.

Let's address the second issue and convert state abbreviation in the work visa dataset to state full name using the list [here](https://raw.githubusercontent.com/sedeh/github.io/master/resources/states.txt).

In [22]:
dsJobsDF3 = dsJobsDF3.dropna(subset=['Employer_State'])
print('dsJobsDF3: {0}'.format(dsJobsDF3.shape))
response = urlopen('https://raw.githubusercontent.com/sedeh/github.io/master/resources/states.txt')
lines = response.readlines()
state_names_dict = {}
for line in lines:
    state_code, state_name = line.decode().split(":")
    state_names_dict[state_code.strip()] = state_name.strip()

state_names_list = []

for index, row in dsJobsDF3.iterrows():
    try:
        state_names_list.append(state_names_dict[row.Work_State])
    except KeyError:
        state_names_list.append(state_names_dict[row.Employer_State])
dsJobsDF4 = dsJobsDF3
dsJobsDF4["Work_State_Code"] = dsJobsDF4.Work_State
dsJobsDF4["Work_State"] = state_names_list

dsJobsDF3: (208595, 17)


We can now remove the extraneous values in the price parity data, `maximum`, `minimum`, `range`, and `nan`.

In [23]:
parityDF2 = parityDF[:-5]
parityDF2 = parityDF2.dropna(subset=['State'])
print('parityDF2: {0}'.format(parityDF2.shape))

parityDF2: (51, 2)


Now we can merge the parity and work visa datasets.

In [24]:
dsJobsDF5 = dsJobsDF4.merge(parityDF2,how='left', left_on='Work_State', right_on='State')
dsJobsDF5.shape

(208595, 20)

We're ready to use the price parity information to adjust salary to reflect regional cost of living differences. Notice that not all states have price parity information. You can see this more clearly below with the presence of `nan` in the `Price_Deflator` column. We should replace those `nan` values with **100.0** so the salary for those states will remain unchanged after adjusting for cost of living differences. 

In [25]:
print('nan present')
print(dsJobsDF5.Price_Deflator.unique())
print("\n")
where_are_NaNs = np.isnan(dsJobsDF5.Price_Deflator)
dsJobsDF5.Price_Deflator[where_are_NaNs] = 100.0
print('nan replaced with 100.0')
print(dsJobsDF5.Price_Deflator.unique())

nan present
[ 102.9    93.65  105.35  107.5   104.95  122.75   94.9   122.1    97.3
  120.     95.8   110.    114.1    99.2   108.65  104.05   97.65  109.8
   97.9   118.1    95.35     nan   96.6   115.95  100.6   103.8   104.8
  112.75  101.3   104.9   124.35   94.75   96.4   108.25   99.3    93.3
  100.3   105.15   97.35   96.05  104.4    96.65  113.15   93.95   92.3
   97.1   103.6   102.3   107.     94.3 ]


nan replaced with 100.0
[ 102.9    93.65  105.35  107.5   104.95  122.75   94.9   122.1    97.3
  120.     95.8   110.    114.1    99.2   108.65  104.05   97.65  109.8
   97.9   118.1    95.35  100.     96.6   115.95  100.6   103.8   104.8
  112.75  101.3   104.9   124.35   94.75   96.4   108.25   99.3    93.3
  100.3   105.15   97.35   96.05  104.4    96.65  113.15   93.95   92.3
   97.1   103.6   102.3   107.     94.3 ]


We're ready to adjust salary with the price parity information. This isn't perfect because we're using price parity information from **2012 and 2013** to adjust salary information from **2002 to 2016**. However, it seems likely that inflation shows consistent trend from year to year among states. For e.g., New York has been more expensive to live in than Iowa for as long as we can remember.

In [26]:
start_time = time.time()

salary_offered_adjusted = []
salary_prevailing_adjusted = []

for index, row in dsJobsDF5.iterrows():
    salary_offered_adjusted.append(row.loc['Salary_Offered'] / (row.loc['Price_Deflator'] / 100))
    salary_prevailing_adjusted.append(row.loc['Salary_Prevailing'] / (row.loc['Price_Deflator'] / 100))
    
dsJobsDF6 = dsJobsDF5
dsJobsDF6["Offered_Salary_Adjusted"] = salary_offered_adjusted
dsJobsDF6["Prevailing_Salary_Adjusted"] = salary_prevailing_adjusted

print("Time to adjust salary for inflation --- %s seconds ---" % (time.time() - start_time))

Time to adjust salary for inflation --- 43.80738282203674 seconds ---


## Add population information

Simply knowing that California has **30,000** data science related jobs compared to say **3500** for Alaska really does not tell us all that much. Relative truth is better where we get a sense of the number of H1B jobs per capita. So let's add population information from census.gov to help us do that. 

In [27]:
start_time = time.time()

url = 'http://www.census.gov/popest/data/state/totals/2015/tables/NST-EST2015-01.xlsx'
response = urlopen(url)
xl = pd.ExcelFile(response)
sheet_names = xl.sheet_names
DF = xl.parse(sheet_names[0], skiprows=8)
DF.columns = ["State", "Census", "Estimates Base", "2010", "2011", "2012", "2013", "2014", "Census_2015"]
censusDF = DF[["State", "Census_2015"]]
censusDF = censusDF.iloc[0:51,]
print(censusDF.head())
print("\n")

# Remove dot sign from the State column
censusDF['State'] = censusDF['State'].map(lambda x: re.sub('\.', '', x))
censusDF['State'] = censusDF['State'].str.title()

# Add population of US territories not included in the data source
temp = pd.DataFrame([['Puerto Rico', 3474182],
                   ['Guam', 159358],
                   ['Virgin Islands', 106405],
                   ['American Samoa', 55519],
                   ['Northern Mariana Islands', 53883],
                   ['Palau', 20918],
                   ['Federated States Of Micronesia', 103549],
                   ['Marshall Islands', 52634]], columns=["State", "Census_2015"])
censusDF = censusDF.append(temp, ignore_index=True)
print(censusDF.head())
print("\n")
print("censusDF: {0}".format(censusDF.shape))
print("Time to read and clean census data --- %s seconds ---" % (time.time() - start_time))

         State  Census_2015
0     .Alabama    4858979.0
1      .Alaska     738432.0
2     .Arizona    6828065.0
3    .Arkansas    2978204.0
4  .California   39144818.0


        State  Census_2015
0     Alabama    4858979.0
1      Alaska     738432.0
2     Arizona    6828065.0
3    Arkansas    2978204.0
4  California   39144818.0


censusDF: (59, 2)
Time to read and clean census data --- 0.2861607074737549 seconds ---


Now we can add the census data to the job data.

In [28]:
dsJobsDF7 = dsJobsDF6.merge(censusDF,how='left', left_on='Work_State', right_on='State')
dsJobsDF7.shape

(208595, 24)

## Prune data

At this point, we can drop columns we clearly don't need. Here are the columns we have:

In [29]:
dsJobsDF7.columns

Index(['Submitted_Date', 'Case_Number', 'Employer_Name', 'Employer_City',
       'Employer_State', 'Employer_Postal_Code', 'Job_Title',
       'Approval_Status', 'Wage_Rate', 'Wage_Rate_Unit', 'Part_Time',
       'Work_City', 'Work_State', 'Prevailing_Wage', 'Job_Title_revised',
       'Salary_Offered', 'Salary_Prevailing', 'Work_State_Code', 'State_x',
       'Price_Deflator', 'Offered_Salary_Adjusted',
       'Prevailing_Salary_Adjusted', 'State_y', 'Census_2015'],
      dtype='object')

In [30]:
dsJobsDF_final = dsJobsDF7.drop(dsJobsDF7.columns[[1,3,4,5,6,7,8,9,10,13,15,16,18,22]], axis=1)
dsJobsDF_final.rename(columns={'Job_Title_revised':'Job_Category'}, inplace=True)

Here's our final dataset and the retained columns. 

In [31]:
print(dsJobsDF_final.columns)
print("\n")
print("dsJobsDF_final has {0} rows and {1} columns".format(dsJobsDF_final.shape[0], dsJobsDF_final.shape[1]))

Index(['Submitted_Date', 'Employer_Name', 'Work_City', 'Work_State',
       'Job_Category', 'Work_State_Code', 'Price_Deflator',
       'Offered_Salary_Adjusted', 'Prevailing_Salary_Adjusted', 'Census_2015'],
      dtype='object')


dsJobsDF_final has 208595 rows and 10 columns


## Persist data

The data is now ready for prime time! We're going to use it to explore [Where the data science jobs are (part2)](https://github.com/sedeh/github.io/blob/master/projects/where_the_data_science_related_jobs_are_part2.ipynb). Before we save the data, let's quickly convert the date string in `Submitted_Date` column to `datetime` and then we can save away.

In [32]:
print(type(dsJobsDF_final.Submitted_Date[0]))
s = dsJobsDF_final["Submitted_Date"]
ts = pd.Series([pd.to_datetime(date_string) for date_string in s])
dsJobsDF_final["Submitted_Date"] = ts
print(type(dsJobsDF_final.Submitted_Date[0]))

<class 'str'>
<class 'pandas.tslib.Timestamp'>


In [33]:
dsJobsDF_final.to_csv('dataScienceJobs.csv', index=False)

In [34]:
print("Total time --- %s minutes ---" % ((time.time() - total_time) / 60.0))

Total time --- 8.932382055123647 minutes ---
